In [18]:
import pandas as pd
import numpy as np


In [19]:
data = pd.read_csv("./NaiveBayes.csv")
data.describe()

,Trial,Mean,Variance,Skewness
count,12.000000,12.000000,12.000000,12.000000
mean,6.500000,0.535833,0.492500,0.382500
std,3.605551,0.200883,0.213121,0.127002
min,1.000000,0.240000,0.210000,0.110000
25%,3.750000,0.337500,0.392500,0.320000
50%,6.500000,0.555000,0.445000,0.420000
75%,9.250000,0.632500,0.542500,0.472500
max,12.000000,0.870000,0.920000,0.520000


In [20]:
features = data[["Mean", "Variance", "Skewness"]]
target = data["Activity"]


In [21]:

class_counts = target.value_counts()
class_probs = class_counts / len(data)

In [22]:
def mean_prob(feature_value, class_data, prior_prob):
  class_mean = class_data["Mean"].mean()
  std_dev = class_data["Mean"].std()
  pdf = 1 / (std_dev * np.sqrt(2 * np.pi)) * np.exp(-(feature_value - class_mean)**2 / (2 * std_dev**2))
  return pdf * prior_prob


In [23]:
def variance_prob(feature_value, class_data, prior_prob):
  class_variance = class_data["Variance"].mean()
  gamma = (class_variance + feature_value) / (2 * class_variance)
  pdf = gamma**(class_variance - 1) * np.exp(-gamma)
  return pdf * prior_prob

In [24]:
def skewness_prob(feature_value, class_data, prior_prob):

  min_skewness = class_data["Skewness"].min()
  max_skewness = class_data["Skewness"].max()
  if min_skewness <= feature_value <= max_skewness:
    pdf = 1 / (max_skewness - min_skewness)
  else:
    pdf = 0
  return pdf * prior_prob

In [25]:
def predict(mean, variance, skewness):
  feature_probs = {
    "RMI": mean_prob(mean, data[target == "RMI"], class_probs["RMI"]),
    "MA": mean_prob(mean, data[target == "MA"], class_probs["MA"]),
    "Rest": mean_prob(mean, data[target == "Rest"], class_probs["Rest"]),
    "LMI": mean_prob(mean, data[target == "LMI"], class_probs["LMI"]),
  }
  feature_probs["MA"] *= variance_prob(variance, data[target == "MA"], class_probs["MA"])
  feature_probs["Rest"] *= variance_prob(variance, data[target == "Rest"], class_probs["Rest"])
  
  feature_probs["RMI"] *= skewness_prob(skewness, data[target == "RMI"], class_probs["RMI"])
  feature_probs["LMI"] *= skewness_prob(skewness, data[target == "LMI"], class_probs["LMI"])

  highest_prob = max(feature_probs.values())
  predicted_activity = max(feature_probs, key=feature_probs.get)

  return predicted_activity

In [27]:
test_case_1 = [0.43, 0.44, 0.61]
test_case_2 = [0.45, 0.32, 0.32]

print(f"Test case 1: {predict(*test_case_1)}")
print(f"Test case 2: {predict(*test_case_2)}")


Test case 1: MA
Test case 2: LMI
